In [2]:
import pymysql
import pandas as pd

In [50]:
print("Starting Data Loading")
data = pd.read_csv("filtered_data.csv")

conn = pymysql.connect(host='localhost',
                    user='root',
                    password='huahua20101',
                    db='song_lyrics')

cursor = conn.cursor()
print("Connected")


Starting Data Loading
Connected


In [51]:
try:
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS songs_data (
        id INT PRIMARY KEY,
        title TEXT,
        genre NVARCHAR(10),
        artist TEXT,
        year YEAR,
        views INT,
        features TEXT,
        lyrics TEXT
        )
        """
    )
    print("Table Created or already exists")
except pymysql.MySQLError as e:
    print(f"Error creating table: {e}")

Table Created or already exists


In [52]:
from tqdm import tqdm

try:
    for row in tqdm(data.itertuples()):
        cursor.execute(
            """
            INSERT INTO songs_data (id, title, genre, artist, year, views, features, lyrics)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (row.id, row.title, row.tag, row.artist, row.year, row.views, row.features, row.lyrics)
        )
    conn.commit()
    print("Data inserted successfully.")
except pymysql.MySQLError as e:
    print(f"Error inserting data: {e}")

5059it [00:00, 6039.84it/s]

Error inserting data: (1406, "Data too long for column 'artist' at row 1")


In [45]:
batch_size = 100
try:
    for i in tqdm(range(0, len(data), batch_size), desc="Inserting Batches", unit="batch"):
        batch = data.iloc[i:i + batch_size]
        values = [
            (row.id, row.title, row.tag, row.artist, row.year, row.views, row.features, row.lyrics)
            for row in batch.itertuples()
        ]
        cursor.executemany(
            """
            INSERT INTO songs_data (id, title, genre, artist, year, views, features, lyrics)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """, values
        )
        conn.commit()
    print("Data inserted successfully.")
except pymysql.MySQLError as e:
    print(f"Error inserting data: {e}")


Inserting Batches:   0%|          | 0/5432 [00:00<?, ?batch/s]

Error inserting data: Cursor closed


In [19]:
def query_sql(query):
    cursor.execute(query)
    data = cursor.fetchall()
    columns = [i[0] for i in cursor.description]
    return pd.DataFrame(data, columns = columns)

In [35]:
query_sql("SELECT * FROM songs_data")

,id,title,genre,artist,year,views,features,lyrics
0,1,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki..."
1,3,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,..."
2,4,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...
3,5,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...
4,6,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ..."
...,...,...,...,...,...,...,...,...
295,12806,Drama A/T,rap,Canibus,2002,2355,{Luminati},[Verse 1: Canibus]\nExecutive Order 11002\nThe...
296,17475,Blackout,rap,DMX,1998,54163,"{JAY-Z,""The LOX""}",[Intro: Sheek Louch & Jadakiss]\nJay\nFuck tha...
297,17476,Keep Your Shit the Hardest,rap,DMX,1998,8212,{},[Chorus]\nNow what y'all want me to do\nKeep y...
298,17565,Still Po Pimpin,rap,Do or Die,1998,9035,"{Twista,""Johnny P""}","[Johnny P]\nDo you wanna ride with me (yeah, o..."


In [41]:
conn.rollback()

In [49]:
cursor.close()
conn.close()
